In [ ]:
# import all needed libraries here:
import gzip
import json
import pandas as pd
import numpy as np
import pickle
import json
from tqdm import tqdm
from re import escape

In [ ]:
# Run this cell to mount the data drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Helper functions

In [ ]:
def parse(path, read_meta_2014=False, fields_to_extract=None):
    g = gzip.open(path, 'rb')
    for l in g:
        data = eval(l) if read_meta_2014 else json.loads(l.strip())
        if fields_to_extract:
            # Filter the data to include only specified fields
            data = {key: data[key] for key in fields_to_extract if key in data}
        yield data

def read_data_from_gzip(path, fields_to_extract=None, read_meta_2014=False):
    i = 0
    df = {}
    for d in tqdm(parse(path, read_meta_2014, fields_to_extract)):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [ ]:
def print_basic_data_stats(data):
  print("Data shape:")
  print(data.shape)
  print("\nMissing values in mergeddata:")
  print(data.isna().sum())
  print("\nPercentage of missing values in metadata:")
  missing_per_column = data.isnull().sum() / len(data) * 100
  print(missing_per_column)

In [ ]:
# Helper function to compute memory footprint of dataset
def check_memory_usage(df):
  # Check size rawdata
  total_memory = df.memory_usage(deep=True).sum()
  print(f"Total memory usage: {total_memory / (1024 ** 2):.2f} MB")

In [ ]:
def save_dict_to_pickle(dictionary, file_path):
  """Saves a dictionary to a pickle file.

  Args:
    dictionary: The dictionary to save.
    file_path: The path to the pickle file.
  """
  with open(file_path, 'wb') as f:
    pickle.dump(dictionary, f)

# Load Book Review Data

In [ ]:
fields_to_extract_from_review_data = ['reviewerID', 'asin','helpful', 'overall', 'unixReviewTime']
book_review_data = read_data_from_gzip("/content/drive/MyDrive/IR/reviews_Books_5.json.gz", read_meta_2014=False, fields_to_extract=fields_to_extract_from_review_data)

8898041it [03:44, 39596.85it/s]


In [ ]:
book_review_data.head()

,reviewerID,asin,helpful,overall,unixReviewTime
0,A10000012B7CGYKOMPQ4L,000100039X,"[0, 0]",5.0,1355616000
1,A2S166WSCFIFP5,000100039X,"[0, 2]",5.0,1071100800
2,A1BM81XB4QHOA3,000100039X,"[0, 0]",5.0,1390003200
3,A1MOSTXNIO5MPJ,000100039X,"[0, 0]",5.0,1317081600
4,A2XQ5LZHTD4AFT,000100039X,"[7, 9]",5.0,1033948800


In [ ]:
print_basic_data_stats(book_review_data)

Data shape:
(8898041, 5)

Missing values in mergeddata:
reviewerID        0
asin              0
helpful           0
overall           0
unixReviewTime    0
dtype: int64

Percentage of missing values in metadata:
reviewerID        0.0
asin              0.0
helpful           0.0
overall           0.0
unixReviewTime    0.0
dtype: float64


# Load Book Meta Data

In [ ]:
fields_to_extract_from_meta_data = ["asin", "price", "salesRank", "related", "categories", "title", "description"]
book_meta_data = read_data_from_gzip("/content/drive/MyDrive/IR/meta_Books.json.gz", read_meta_2014=True, fields_to_extract=fields_to_extract_from_meta_data)

2370585it [05:30, 7174.54it/s]


In [ ]:
book_meta_data.head()

,asin,salesRank,categories,title,description,price,related
0,0001048791,{'Books': 6334800},[[Books]],"The Crucible: Performed by Stuart Pankin, Jero...",NaN,NaN,NaN
1,0001048775,{'Books': 13243226},[[Books]],Measure for Measure: Complete &amp; Unabridged,William Shakespeare is widely regarded as the ...,NaN,NaN
2,0001048236,{'Books': 8973864},[[Books]],The Sherlock Holmes Audio Collection,"&#34;One thing is certain, Sherlockians, put a...",9.26,"{'also_viewed': ['1442300191', '9626349786', '..."
3,0000401048,{'Books': 6448843},[[Books]],The rogue of publishers' row;: Confessions of ...,NaN,NaN,{'also_viewed': ['068240103X']}
4,0001019880,{'Books': 9589258},[[Books]],Classic Soul Winner's New Testament Bible,NaN,5.39,"{'also_viewed': ['B003HMB5FC', '0834004593'], ..."


In [ ]:
book_meta_data.tail()

,asin,salesRank,categories,title,description,price,related
2370580,B00M111QCM,NaN,"[[Books, Business & Money, Industries & Profes...",NaN,NaN,NaN,NaN
2370581,B00M0AEPXG,NaN,"[[Books, Politics & Social Sciences, Women's S...",NaN,NaN,NaN,NaN
2370582,B00LZFHL7Y,NaN,"[[Books, Literature & Fiction, Erotica], [Kind...",NaN,NaN,0.99,"{'also_bought': ['B00LXK4KIO', 'B00LZ19FE0', '..."
2370583,B00LV8M74W,NaN,"[[Books, Literature & Fiction, Genre Fiction, ...",NaN,NaN,3.99,"{'also_bought': ['B00LYOC1R6', 'B00L0WXZIG', '..."
2370584,B00LV8STE4,NaN,"[[Books, Arts & Photography, Music, Musical Ge...",NaN,NaN,NaN,NaN


In [ ]:
print_basic_data_stats(book_meta_data)

Data shape:
(2370585, 7)

Missing values in mergeddata:
asin                 0
salesRank       479422
categories           0
title           431818
description    1249227
price           691175
related         750156
dtype: int64

Percentage of missing values in metadata:
asin            0.000000
salesRank      20.223784
categories      0.000000
title          18.215673
description    52.696993
price          29.156305
related        31.644341
dtype: float64


# Load 2018 Book Meta Data

In [ ]:
fields_to_extract_from_meta_data = ["asin", "title", "category"]
meta_data_raw_2018 = read_data_from_gzip("/content/drive/MyDrive/IR/meta_Books_2018.json.gz", read_meta_2014=False, fields_to_extract=fields_to_extract_from_meta_data)

2934949it [01:41, 28887.69it/s]


In [ ]:
meta_data_raw_2018.head()

,asin,title,category,category_1,category_2,category_3,category_4,category_5,category_6,category_7,...,category_9,category_10,category_11,category_12,category_13,category_14,category_15,category_16,category_17,category_18
0,0000092878,Biology Gods Living Creation Third Edition 10 ...,[],None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,000047715X,Mksap 16 Audio Companion: Medical Knowledge Se...,"[Books, New, Used & Rental Textbooks, Medicine...",Books,"New, Used & Rental Textbooks",Medicine & Health Sciences,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,0000004545,"Flex! Discography of North American Punk, Hard...","[Books, Arts & Photography, Music]",Books,Arts & Photography,Music,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,0000013765,Heavenly Highway Hymns: Shaped-Note Hymnal,"[Books, Arts & Photography, Music]",Books,Arts & Photography,Music,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,0000000116,Georgina Goodman Nelson Womens Size 8.5 Purple...,[],None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


# Load 2023 Book Meta Data

In [ ]:
#fields_to_extract_from_meta_data = ["asin", "price", "description", "salesRank", "related"]
fields_to_extract_from_meta_data = ["title", "categories", "parent_asin"]
meta_data_raw_2023 = read_data_from_gzip("/content/drive/MyDrive/IR/meta_Books_2023.jsonl.gz", read_meta_2014=False, fields_to_extract=fields_to_extract_from_meta_data)

4448181it [04:41, 15783.94it/s]


In [ ]:
meta_data_raw_2023 = meta_data_raw_2023.rename(columns={'parent_asin': 'asin'})

In [ ]:
meta_data_raw_2023.head()

,title,categories,asin
0,Chaucer,"[Books, Literature & Fiction, History & Critic...",0701169850
1,Notes from a Kidwatcher,"[Books, Reference, Words, Language & Grammar]",0435088688
2,Service: A Navy SEAL at War,"[Books, Biographies & Memoirs, Leaders & Notab...",0316185361
3,Monstrous Stories #4: The Day the Mice Stood S...,"[Books, Children's Books, Science Fiction & Fa...",0545425573
4,Parker & Knight,"[Books, Mystery, Thriller & Suspense, Thriller...",B00KFOP3RG


# Test some meta data merges merges (Not important!)

In [ ]:
# Perform the initial merge, keeping all rows from book_meta_data
merged_meta_left = pd.merge(book_meta_data, meta_data_raw_2018[['asin', 'title']], on='asin', how='left', suffixes=('', '_raw'))

# Fill missing 'title' values in book_meta_data with values from meta_data_raw_2018
#merged_meta_left['title'] = merged_meta_left['title'].fillna(merged_meta_left['title_raw'])

# Drop the temporary 'title_raw' column
#merged_meta_left = merged_meta_left.drop(columns=['title_raw'])

In [ ]:
merged_meta_left.head()

,asin,salesRank,categories,title,description,price,related,title_raw
0,0001048791,{'Books': 6334800},[[Books]],"The Crucible: Performed by Stuart Pankin, Jero...",NaN,NaN,NaN,NaN
1,0001048775,{'Books': 13243226},[[Books]],Measure for Measure: Complete &amp; Unabridged,William Shakespeare is widely regarded as the ...,NaN,NaN,Measure for Measure Unabridged
2,0001048236,{'Books': 8973864},[[Books]],The Sherlock Holmes Audio Collection,"&#34;One thing is certain, Sherlockians, put a...",9.26,"{'also_viewed': ['1442300191', '9626349786', '...",The Sherlock Holmes Audio Collection
3,0000401048,{'Books': 6448843},[[Books]],The rogue of publishers' row;: Confessions of ...,NaN,NaN,{'also_viewed': ['068240103X']},NaN
4,0001019880,{'Books': 9589258},[[Books]],Classic Soul Winner's New Testament Bible,NaN,5.39,"{'also_viewed': ['B003HMB5FC', '0834004593'], ...",Classic Soul Winner's New Testament Bible


In [ ]:
print_basic_data_stats(merged_meta_left)

Data shape:
(2372747, 7)

Missing values in mergeddata:
asin                 0
salesRank       479665
categories           0
title           396700
description    1251300
price           692992
related         751704
dtype: int64

Percentage of missing values in metadata:
asin            0.000000
salesRank      20.215598
categories      0.000000
title          16.719018
description    52.736343
price          29.206317
related        31.680748
dtype: float64


In [ ]:
# Perform the initial merge, keeping all rows from book_meta_data
merged_meta_left = pd.merge(merged_meta_left, meta_data_raw_2023[['asin', 'title']], on='asin', how='left', suffixes=('', '_raw'))

# Fill missing 'title' values in book_meta_data with values from meta_data_raw_2018
merged_meta_left['title'] = merged_meta_left['title'].fillna(merged_meta_left['title_raw'])

# Drop the temporary 'title_raw' column
merged_meta_left = merged_meta_left.drop(columns=['title_raw'])

In [ ]:
print_basic_data_stats(merged_meta_left)

Data shape:
(2372747, 7)

Missing values in mergeddata:
asin                 0
salesRank       479665
categories           0
title           382340
description    1251300
price           692992
related         751704
dtype: int64

Percentage of missing values in metadata:
asin            0.000000
salesRank      20.215598
categories      0.000000
title          16.113812
description    52.736343
price          29.206317
related        31.680748
dtype: float64


#  Create integer mappings for ```reviewerID``` & ```asin``` columns

In [ ]:
def create_mappings(df):
    """
    Create mappings for a given column in a DataFrame.

    Args:
        df (pd.DataFrame): Input DataFrame.

    Returns:
        user_mapping (dict): Mapping for user IDs.
        item_mapping (dict): Mapping for item IDs.
        reverse_user_mapping (dict): Reverse mapping for user IDs.
        reverse_item_mapping (dict): Reverse mapping for item IDs.

    """
    # Map `reviewerID` and `asin` to unique integer IDs
    user_mapping = {user: idx for idx, user in enumerate(df['reviewerID'].unique())}
    item_mapping = {item: idx for idx, item in enumerate(df['asin'].unique())}

    # Create reverse mappings
    reverse_user_mapping = {idx: user for user, idx in user_mapping.items()}
    reverse_item_mapping = {idx: item for item, idx in item_mapping.items()}


    return user_mapping, item_mapping, reverse_user_mapping, reverse_item_mapping

In [ ]:
user_mapping, item_mapping, reverse_user_mapping, reverse_item_mapping = create_mappings(book_review_data)

In [ ]:
# Optional run this code to save mappings for reuse later:
save_dict_to_pickle(user_mapping, '/content/drive/MyDrive/IR/user_mapping.pkl')
save_dict_to_pickle(item_mapping, '/content/drive/MyDrive/IR/item_mapping.pkl')
save_dict_to_pickle(reverse_user_mapping, '/content/drive/MyDrive/IR/reverse_user_mapping.pkl')
save_dict_to_pickle(reverse_item_mapping, '/content/drive/MyDrive/IR/reverse_item_mapping.pkl')

In [ ]:
# Add the mappings to the review data
book_review_data['user_id'] = book_review_data['reviewerID'].map(user_mapping)
book_review_data['item_id'] = book_review_data['asin'].map(item_mapping)

In [ ]:
book_review_data.head()

,reviewerID,asin,helpful,overall,unixReviewTime,user_id,item_id
0,A10000012B7CGYKOMPQ4L,000100039X,"[0, 0]",5.0,1355616000,0,0
1,A2S166WSCFIFP5,000100039X,"[0, 2]",5.0,1071100800,1,0
2,A1BM81XB4QHOA3,000100039X,"[0, 0]",5.0,1390003200,2,0
3,A1MOSTXNIO5MPJ,000100039X,"[0, 0]",5.0,1317081600,3,0
4,A2XQ5LZHTD4AFT,000100039X,"[7, 9]",5.0,1033948800,4,0


# Feature Engineering for Book Review Data

## Aggregate user behavior to compute average rating and total review statistics

In [ ]:
# Aggregate user behavior to compute average rating and total review statistics
user_stats = book_review_data.groupby('user_id').agg(
    total_user_reviews=('asin', 'count'),
    average_rating=('overall', 'mean')
).reset_index()


book_review_data = pd.merge(book_review_data, user_stats, on='user_id', how='left')

## Seperate helpful into helpful_votes and total_votes columns

In [ ]:
def process_helpful_column(df):
    """
    Process the 'helpful' column to extract 'helpful_votes' and 'total_votes',
    handling edge cases like NaN, invalid types, or inconsistent lengths.
    """
    # Ensure the 'helpful' column is consistent by replacing invalid types or lengths with [0, 0]
    df['helpful'] = df['helpful'].map(
        lambda x: x if isinstance(x, (list, tuple)) and len(x) == 2 else [0, 0]
    )

    # Extract votes using vectorized operations
    helpful_array = np.array(df['helpful'].tolist())
    df['helpful_votes'] = helpful_array[:, 0].astype(int)
    df['total_votes'] = helpful_array[:, 1].astype(int)

    # Drop the original column
    return df.drop(columns=['helpful'])


In [ ]:
book_review_data = process_helpful_column(book_review_data)

In [ ]:
book_review_data.head()

,reviewerID,asin,overall,unixReviewTime,user_id,item_id,total_user_reviews,average_rating,helpful_votes,total_votes
0,A10000012B7CGYKOMPQ4L,000100039X,5.0,1355616000,0,0,8,4.375000,0,0
1,A2S166WSCFIFP5,000100039X,5.0,1071100800,1,0,556,4.201439,0,2
2,A1BM81XB4QHOA3,000100039X,5.0,1390003200,2,0,6,4.333333,0,0
3,A1MOSTXNIO5MPJ,000100039X,5.0,1317081600,3,0,9,4.000000,0,0
4,A2XQ5LZHTD4AFT,000100039X,5.0,1033948800,4,0,76,3.750000,7,9


# Feature Engineering for Book Meta Data (2014)

## Unpack the salesRank column

In [ ]:
def process_sales_rank_column(df):
    """
    Process the 'salesRank' column to extract the 'books_rank', handling cases
    where the column contains invalid types or missing data.
    """
    df['books_rank'] = pd.to_numeric(
        df['salesRank'].map(lambda x: x.get('Books') if isinstance(x, dict) else None),
        errors='coerce'
    ).astype('Int64')

    return df.drop(columns=['salesRank'])

In [ ]:
book_meta_data = process_sales_rank_column(book_meta_data)

In [ ]:
book_meta_data.head()

,asin,categories,title,description,price,related,books_rank
0,0001048791,[[Books]],"The Crucible: Performed by Stuart Pankin, Jero...",NaN,NaN,NaN,6334800
1,0001048775,[[Books]],Measure for Measure: Complete &amp; Unabridged,William Shakespeare is widely regarded as the ...,NaN,NaN,13243226
2,0001048236,[[Books]],The Sherlock Holmes Audio Collection,"&#34;One thing is certain, Sherlockians, put a...",9.26,"{'also_viewed': ['1442300191', '9626349786', '...",8973864
3,0000401048,[[Books]],The rogue of publishers' row;: Confessions of ...,NaN,NaN,{'also_viewed': ['068240103X']},6448843
4,0001019880,[[Books]],Classic Soul Winner's New Testament Bible,NaN,5.39,"{'also_viewed': ['B003HMB5FC', '0834004593'], ...",9589258


## Unpack the related column

In [ ]:
def process_related_column(df):
    """
    Process the 'related' column to extract 'also_bought', 'bought_together', and 'buy_after_viewing',
    setting missing or empty values to None.
    """
    # Ensure the 'related' column contains valid dictionaries or empty ones
    df['related'] = df['related'].map(lambda x: x if isinstance(x, dict) else {})

    # Extract relevant keys and set None if the value is missing or empty
    df['related_also_bought'] = df['related'].map(lambda x: x.get('also_bought') or None)
    df['related_bought_together'] = df['related'].map(lambda x: x.get('bought_together') or None)
    df['related_bought_after_viewing'] = df['related'].map(lambda x: x.get('buy_after_viewing') or None)

    # Drop the original column
    return df.drop(columns=['related'])

In [ ]:
book_meta_data = process_related_column(book_meta_data)

In [ ]:
book_meta_data.head()

,asin,categories,title,description,price,books_rank,related_also_bought,related_bought_together,related_bought_after_viewing
0,0001048791,[[Books]],"The Crucible: Performed by Stuart Pankin, Jero...",NaN,NaN,6334800,None,None,None
1,0001048775,[[Books]],Measure for Measure: Complete &amp; Unabridged,William Shakespeare is widely regarded as the ...,NaN,13243226,None,None,None
2,0001048236,[[Books]],The Sherlock Holmes Audio Collection,"&#34;One thing is certain, Sherlockians, put a...",9.26,8973864,None,None,[0312089457]
3,0000401048,[[Books]],The rogue of publishers' row;: Confessions of ...,NaN,NaN,6448843,None,None,None
4,0001019880,[[Books]],Classic Soul Winner's New Testament Bible,NaN,5.39,9589258,None,None,"[031095360X, 0834004577]"


## Unpack the categories column

The categories column in the 2014 metadata contains a list containing sublists. We are only interested in extracting the first sublist.

In [ ]:
def print_value_counts(df, column):
  # Set option to display all rows
  pd.set_option("display.max_rows", None)

  # Get and print value counts
  category_counts = df[column].value_counts()
  print(category_counts)

  pd.reset_option("display.max_rows")

In [ ]:
# Extract only the first sublist in each list
book_meta_data['first_sublist'] = book_meta_data['categories'].apply(lambda x: x[0])

# Determine the maximum length of the first sublist
max_len = book_meta_data['first_sublist'].apply(len).max()

In [ ]:
book_meta_data.head()

,asin,categories,title,description,price,books_rank,related_also_bought,related_bought_together,related_bought_after_viewing,first_sublist
0,0001048791,[[Books]],"The Crucible: Performed by Stuart Pankin, Jero...",NaN,NaN,6334800,None,None,None,[Books]
1,0001048775,[[Books]],Measure for Measure: Complete &amp; Unabridged,William Shakespeare is widely regarded as the ...,NaN,13243226,None,None,None,[Books]
2,0001048236,[[Books]],The Sherlock Holmes Audio Collection,"&#34;One thing is certain, Sherlockians, put a...",9.26,8973864,None,None,[0312089457],[Books]
3,0000401048,[[Books]],The rogue of publishers' row;: Confessions of ...,NaN,NaN,6448843,None,None,None,[Books]
4,0001019880,[[Books]],Classic Soul Winner's New Testament Bible,NaN,5.39,9589258,None,None,"[031095360X, 0834004577]",[Books]


In [ ]:
book_meta_data.tail()

,asin,categories,title,description,price,books_rank,related_also_bought,related_bought_together,related_bought_after_viewing,first_sublist
2370580,B00M111QCM,"[[Books, Business & Money, Industries & Profes...",NaN,NaN,NaN,<NA>,None,None,None,"[Books, Business & Money, Industries & Profess..."
2370581,B00M0AEPXG,"[[Books, Politics & Social Sciences, Women's S...",NaN,NaN,NaN,<NA>,None,None,None,"[Books, Politics & Social Sciences, Women's St..."
2370582,B00LZFHL7Y,"[[Books, Literature & Fiction, Erotica], [Kind...",NaN,NaN,0.99,<NA>,"[B00LXK4KIO, B00LZ19FE0, B00LZ4D0UC, B00LNQ27T...",None,"[B00LDD8NDO, B00KLBHX44, B00JS9YJPQ, B00LADEH0U]","[Books, Literature & Fiction, Erotica]"
2370583,B00LV8M74W,"[[Books, Literature & Fiction, Genre Fiction, ...",NaN,NaN,3.99,<NA>,"[B00LYOC1R6, B00L0WXZIG, B00JCZSTDE, B00JWGG0Q...",None,[B004Q3RTXS],"[Books, Literature & Fiction, Genre Fiction, M..."
2370584,B00LV8STE4,"[[Books, Arts & Photography, Music, Musical Ge...",NaN,NaN,NaN,<NA>,None,None,None,"[Books, Arts & Photography, Music, Musical Gen..."


In [ ]:
print_value_counts(book_meta_data, 'first_sublist')

first_sublist
[Books]                                                                                                                                    1940253
[Books, Literature & Fiction]                                                                                                                74612
[Books, Literature & Fiction, Erotica]                                                                                                       29426
[Books, Literature & Fiction, Contemporary]                                                                                                  15399
[Books, Literature & Fiction, Genre Fiction, Action & Adventure]                                                                             13416
[Books, Gay & Lesbian, Literature & Fiction, Fiction, Gay]                                                                                    8018
[Books, Literature & Fiction, Genre Fiction, Horror]                                                    

## Unpack the first_sublist column

We want to unpack these extracted sublists even more to extract the actual book categories. For example, from this sublist, ```[Books, Business & Money, Skills, Time Management]```, we want to extract Business & Money as it is the main book category of this item.

In [ ]:
# Create new column names
new_cols = [f'category_{i+1}' for i in range(max_len)]

# Unpack the first sublist into separate columns
book_meta_data[new_cols] = pd.DataFrame(book_meta_data['first_sublist'].tolist(), index=book_meta_data.index)

# Drop unnecessary columns if needed
book_meta_data = book_meta_data.drop(columns=['categories', 'first_sublist'])

In [ ]:
book_meta_data.head()

,asin,title,description,price,books_rank,related_also_bought,related_bought_together,related_bought_after_viewing,category_1,category_2,category_3,category_4,category_5,category_6,category_7
0,0001048791,"The Crucible: Performed by Stuart Pankin, Jero...",NaN,NaN,6334800,None,None,None,Books,None,None,None,None,None,None
1,0001048775,Measure for Measure: Complete &amp; Unabridged,William Shakespeare is widely regarded as the ...,NaN,13243226,None,None,None,Books,None,None,None,None,None,None
2,0001048236,The Sherlock Holmes Audio Collection,"&#34;One thing is certain, Sherlockians, put a...",9.26,8973864,None,None,[0312089457],Books,None,None,None,None,None,None
3,0000401048,The rogue of publishers' row;: Confessions of ...,NaN,NaN,6448843,None,None,None,Books,None,None,None,None,None,None
4,0001019880,Classic Soul Winner's New Testament Bible,NaN,5.39,9589258,None,None,"[031095360X, 0834004577]",Books,None,None,None,None,None,None


In [ ]:
book_meta_data.tail()

,asin,title,description,price,books_rank,related_also_bought,related_bought_together,related_bought_after_viewing,category_1,category_2,category_3,category_4,category_5,category_6,category_7
2370580,B00M111QCM,NaN,NaN,NaN,<NA>,None,None,None,Books,Business & Money,Industries & Professions,Human Resources & Personnel Management,None,None,None
2370581,B00M0AEPXG,NaN,NaN,NaN,<NA>,None,None,None,Books,Politics & Social Sciences,Women's Studies,None,None,None,None
2370582,B00LZFHL7Y,NaN,NaN,0.99,<NA>,"[B00LXK4KIO, B00LZ19FE0, B00LZ4D0UC, B00LNQ27T...",None,"[B00LDD8NDO, B00KLBHX44, B00JS9YJPQ, B00LADEH0U]",Books,Literature & Fiction,Erotica,None,None,None,None
2370583,B00LV8M74W,NaN,NaN,3.99,<NA>,"[B00LYOC1R6, B00L0WXZIG, B00JCZSTDE, B00JWGG0Q...",None,[B004Q3RTXS],Books,Literature & Fiction,Genre Fiction,Medical,None,None,None
2370584,B00LV8STE4,NaN,NaN,NaN,<NA>,None,None,None,Books,Arts & Photography,Music,Musical Genres,Jazz,None,None


### Here we are just inspecting the value counts of all the unpacked variables.

In [ ]:
print_value_counts(book_meta_data, 'category_1')

category_1
Books                    2369971
Arts, Crafts & Sewing        610
Baby Products                  3
Automotive                     1
Name: count, dtype: int64


In [ ]:
print_value_counts(book_meta_data, 'category_2') # This is the main one we want!

category_2
Literature & Fiction            200791
Children's Books                 25424
Christian Books & Bibles         22470
Business & Money                 18174
Biographies & Memoirs            18145
Health, Fitness & Dieting        18036
Cookbooks, Food & Wine           11678
Gay & Lesbian                    11533
History                          10539
Arts & Photography                9340
Humor & Entertainment             8781
Religion & Spirituality           8696
Self-Help                         8188
Crafts, Hobbies & Home            7988
Reference                         6831
Education & Teaching              6092
Computers & Technology            5031
Politics & Social Sciences        4595
Parenting & Relationships         4095
Comics & Graphic Novels           3547
Travel                            3421
Teen & Young Adult                3349
Sports & Outdoors                 3309
Science & Math                    2160
Engineering & Transportation      2026
Medical Books 

In [ ]:
print_value_counts(book_meta_data, 'category_3')

category_3
Genre Fiction                                44502
Erotica                                      29426
Literature & Fiction                         18758
Contemporary                                 15418
Humor                                        10349
Christian Living                              6469
United States                                 6225
Short Stories                                 6083
Animals                                       5010
Women's Fiction                               4631
Schools & Teaching                            4256
Classics                                      3998
Memoirs                                       3879
Alternative Medicine                          3877
Poetry                                        3567
Americas                                      3525
Bible Study & Reference                       3335
Diets & Weight Loss                           3320
Science Fiction & Fantasy                     3249
Growing Up & Facts o

In [ ]:
print_value_counts(book_meta_data, 'category_4')

category_4
Horror                                                           14454
Action & Adventure                                               13435
Fiction                                                           9663
Historical                                                        7430
Single Author                                                     5451
United States                                                     4548
Contemporary Women                                                4036
African American                                                  3964
General                                                           3824
Coming of Age                                                     3465
Romance                                                           2453
Dogs                                                              2066
Erotica                                                           1887
Linguistics                                                       

In [ ]:
print_value_counts(book_meta_data, 'category_5')

category_5
Gay                                         8026
Occult                                      3811
Ghosts                                      1976
Romance                                     1879
Dark Fantasy                                1491
Lesbian                                     1462
General                                     1328
Urban                                       1194
State & Local                                976
Sleep                                        907
New Testament                                721
Contemporary                                 650
Breeds                                       602
Old Testament                                512
Civil War                                    490
Cleaning, Caretaking & Relocating            452
Christian                                    440
Training                                     417
Military                                     407
Guides                                       393
History  

In [ ]:
print_value_counts(book_meta_data, 'category_6')

category_6
Decorating                     84
Campaigns & Battlefields       80
1800s                          40
Stories                        40
Neuroscience                   36
Projects                       36
Diseases                       35
Westerns                       34
Confederacy                    34
London                         33
Christian                      26
Windows 8                      25
Violins                        21
Hymns                          21
Colonial                       20
Ethnic                         18
Jewish                         17
Iraq War                       16
Clinical Chemistry             15
1900s                          14
Reference & Interpretation     14
Ukuleles                       14
Abolition                      13
Communicable Diseases          11
Alzheimer's Disease            11
Pollution                      11
C++                            11
Diagnostic Imaging             10
C                              10
Pet

In [ ]:
print_value_counts(book_meta_data, 'category_7')

category_7
Gettysburg        47
Antietam           9
Bull Run           8
Fredericksburg     5
AIDS & HIV         4
Appomattox         3
Viral              1
Psoriasis          1
Name: count, dtype: int64


In [ ]:
print_basic_data_stats(book_meta_data)

Data shape:
(2370585, 15)

Missing values in mergeddata:
asin                                  0
title                            431818
description                     1249227
price                            691175
books_rank                       479568
related_also_bought             1101588
related_bought_together         1823156
related_bought_after_viewing    1125260
category_1                            0
category_2                      1940253
category_3                      2030516
category_4                      2142839
category_5                      2319613
category_6                      2369607
category_7                      2370507
dtype: int64

Percentage of missing values in metadata:
asin                             0.000000
title                           18.215673
description                     52.696993
price                           29.156305
books_rank                      20.229943
related_also_bought             46.469036
related_bought_together         76.

## Drop unneeded category columns

In [ ]:
# we only wante to keep category_2 so we drop the rest
book_meta_data = book_meta_data.drop(columns=['category_1', 'category_3', 'category_4', 'category_5', 'category_6', 'category_7'])

In [ ]:
book_meta_data = book_meta_data.rename(columns={'category_2': 'category'})

In [ ]:
check_memory_usage(book_meta_data)

Total memory usage: 2201.75 MB


In [ ]:
print_basic_data_stats(book_meta_data)

Data shape:
(2370585, 9)

Missing values in mergeddata:
asin                                  0
title                            431818
description                     1249227
price                            691175
books_rank                       479568
related_also_bought             1101588
related_bought_together         1823156
related_bought_after_viewing    1125260
category                        1940253
dtype: int64

Percentage of missing values in metadata:
asin                             0.000000
title                           18.215673
description                     52.696993
price                           29.156305
books_rank                      20.229943
related_also_bought             46.469036
related_bought_together         76.907430
related_bought_after_viewing    47.467608
category                        81.847012
dtype: float64


As can be seen in the overview above over 81% of te book categories are missing in the 2014 metadata. This is a bit of an issue, since we need them for the Categorical Alignment Loss. Therefore, we need to take some creative liberty and attempt to aquire these missing values through other means.

Therefore, I have decided to get these from the other available metadatasets, namely, book metadata 2018, and book metadata 2023

Meta Data 2018

We apply similar extraction steps as before, but this time around the object contained inside the category column is slightly different, its a list containing multiple string values such as: Books, Main category, sub category, etc. So once again we are interested in the second entry of the list.

In [ ]:
# Get the maximum length of the lists in the 'category' column
max_len = meta_data_raw_2018['category'].apply(len).max()

# Create new column names based on the maximum length
new_cols = [f'category_{i+1}' for i in range(max_len)]

# Unpack the lists into separate columns
meta_data_raw_2018[new_cols] = pd.DataFrame(meta_data_raw_2018['category'].values.tolist(), index= meta_data_raw_2018.index)

In [ ]:
meta_data_raw_2018.head()

,asin,title,category,category_1,category_2,category_3,category_4,category_5,category_6,category_7,...,category_9,category_10,category_11,category_12,category_13,category_14,category_15,category_16,category_17,category_18
0,0000092878,Biology Gods Living Creation Third Edition 10 ...,[],None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,000047715X,Mksap 16 Audio Companion: Medical Knowledge Se...,"[Books, New, Used & Rental Textbooks, Medicine...",Books,"New, Used & Rental Textbooks",Medicine & Health Sciences,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,0000004545,"Flex! Discography of North American Punk, Hard...","[Books, Arts & Photography, Music]",Books,Arts & Photography,Music,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,0000013765,Heavenly Highway Hymns: Shaped-Note Hymnal,"[Books, Arts & Photography, Music]",Books,Arts & Photography,Music,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,0000000116,Georgina Goodman Nelson Womens Size 8.5 Purple...,[],None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
meta_data_raw_2018.tail()

,asin,title,category,category_1,category_2,category_3,category_4,category_5,category_6,category_7,...,category_9,category_10,category_11,category_12,category_13,category_14,category_15,category_16,category_17,category_18
2934944,B01HJBPTUI,Made Men: A Thriller (Law of Retaliation Book ...,"[Books, Mystery, Thriller & Suspense, Thriller...",Books,"Mystery, Thriller & Suspense",Thrillers & Suspense,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2934945,B01HJC63ZM,Raptor&#39;s Desire (A Planet Desire novelette...,"[Books, Literature & Fiction, Erotica]",Books,Literature & Fiction,Erotica,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2934946,B01HJEB422,"LG K4 Case,LG Optimus Zone 3 Case,LG Spree Cas...",[],None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2934947,B01HJDS76Y,Magickal Incantations,"[Books, Religion & Spirituality, New Age & Spi...",Books,Religion & Spirituality,New Age & Spirituality,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2934948,B01HJFHYMA,White Away EURO 2016 Match Football Soccer Adu...,[],None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
print_value_counts(meta_data_raw_2018, 'category_2')

category_2
Literature &amp; Fiction                          245086
Children's Books                                  241105
History                                           138229
Literature & Fiction                              110284
Arts &amp; Photography                            108042
Christian Books &amp; Bibles                      107721
New, Used &amp; Rental Textbooks                  101555
Reference                                          69446
Biographies &amp; Memoirs                          67136
Crafts, Hobbies &amp; Home                         65481
Business &amp; Money                               64113
Romance                                            58542
Religion &amp; Spirituality                        54373
Self-Help                                          53525
Mystery, Thriller &amp; Suspense                   53009
Arts & Photography                                 50244
Politics &amp; Social Sciences                     48914
Travel              

In [ ]:
meta_data_raw_2018 = meta_data_raw_2018.drop(columns=['category','category_1', 'category_3',
       'category_4', 'category_5', 'category_6', 'category_7', 'category_8',
       'category_9', 'category_10', 'category_11', 'category_12',
       'category_13', 'category_14', 'category_15', 'category_16',
       'category_17', 'category_18'])

In [ ]:
meta_data_raw_2018 = meta_data_raw_2018.rename(columns={'category_2': 'category'})

In [ ]:
print_basic_data_stats(meta_data_raw_2018)

Data shape:
(2934949, 3)

Missing values in mergeddata:
asin             0
title            0
category    389429
dtype: int64

Percentage of missing values in metadata:
asin         0.00000
title        0.00000
category    13.26868
dtype: float64


# Meta Data 2023

We apply the same process to the 2023 metadata to extract the main book categories.


In [ ]:
# Get the maximum length of the lists in the 'category' column
max_len = meta_data_raw_2023['categories'].apply(len).max()

# Create new column names based on the maximum length
new_cols = [f'category_{i+1}' for i in range(max_len)]

# Unpack the lists into separate columns
meta_data_raw_2023[new_cols] = pd.DataFrame(meta_data_raw_2023['categories'].values.tolist(), index= meta_data_raw_2023.index)

In [ ]:
meta_data_raw_2023.head()

,title,categories,asin,category_1,category_2,category_3,category_4,category_5,category_6,category_7
0,Chaucer,"[Books, Literature & Fiction, History & Critic...",0701169850,Books,Literature & Fiction,History & Criticism,None,None,None,None
1,Notes from a Kidwatcher,"[Books, Reference, Words, Language & Grammar]",0435088688,Books,Reference,"Words, Language & Grammar",None,None,None,None
2,Service: A Navy SEAL at War,"[Books, Biographies & Memoirs, Leaders & Notab...",0316185361,Books,Biographies & Memoirs,Leaders & Notable People,None,None,None,None
3,Monstrous Stories #4: The Day the Mice Stood S...,"[Books, Children's Books, Science Fiction & Fa...",0545425573,Books,Children's Books,Science Fiction & Fantasy,None,None,None,None
4,Parker & Knight,"[Books, Mystery, Thriller & Suspense, Thriller...",B00KFOP3RG,Books,"Mystery, Thriller & Suspense",Thrillers & Suspense,None,None,None,None


In [ ]:
meta_data_raw_2023.tail()

,title,categories,asin,category_1,category_2,category_3,category_4,category_5,category_6,category_7
4448176,Please Excuse My Daughter,"[Books, Biographies & Memoirs, Community & Cul...",1594483574,Books,Biographies & Memoirs,Community & Culture,None,None,None,None
4448177,Inside the Southeast Asian Kitchen: Foodlore a...,"[Books, Cookbooks, Food & Wine, Regional & Int...",9719317051,Books,"Cookbooks, Food & Wine",Regional & International,None,None,None,None
4448178,Origin of Negative Dialectics,"[Books, Politics & Social Sciences, Philosophy]",0029051509,Books,Politics & Social Sciences,Philosophy,None,None,None,None
4448179,Trails Illustrated National Parks Guadalupe Mo...,"[Books, Reference, Atlases & Maps]",0925873039,Books,Reference,Atlases & Maps,None,None,None,None
4448180,Engels After Marx,"[Books, New, Used & Rental Textbooks, Social S...",0271024852,Books,"New, Used & Rental Textbooks",Social Sciences,None,None,None,None


In [ ]:
print_value_counts(meta_data_raw_2023, 'category_2')

category_2
Literature & Fiction                                687946
Children's Books                                    433656
Mystery, Thriller & Suspense                        217030
Arts & Photography                                  215562
History                                             181989
Biographies & Memoirs                               149250
Crafts, Hobbies & Home                              146224
Christian Books & Bibles                            145354
Business & Money                                    131002
Politics & Social Sciences                          118903
New, Used & Rental Textbooks                        112050
Science Fiction & Fantasy                           109916
Reference                                            95808
Science & Math                                       92540
Romance                                              90589
Teen & Young Adult                                   85319
Cookbooks, Food & Wine                       

In [ ]:
meta_data_raw_2023 = meta_data_raw_2023.drop(columns=['category_1','category_3', 'category_4', 'category_5', 'category_6', 'category_7'])

In [ ]:
meta_data_raw_2023 = meta_data_raw_2023.rename(columns={'category_2': 'category'})

In [ ]:
print_basic_data_stats(meta_data_raw_2023)

Data shape:
(4448181, 4)

Missing values in mergeddata:
title              0
categories         0
asin               0
category      528673
dtype: int64

Percentage of missing values in metadata:
title          0.00000
categories     0.00000
asin           0.00000
category      11.88515
dtype: float64


# Merge Meta Data Together

In [ ]:
# For some reason the 2018 metadata contains duplicated values unlike the other two metadatasets we have to deal with this.
meta_data_raw_2018[meta_data_raw_2018.duplicated(subset=['asin'], keep=False)]

In [ ]:
meta_data_raw_2018[meta_data_raw_2018["asin"] == "B00000IJYC"]

,asin,title,category
2603424,B00000IJYC,Adventure Journal,None
2608349,B00000IJYC,Adventure Journal,None


In [ ]:
# To verify the 2014 metadata has no duplicates!
book_meta_data[book_meta_data.duplicated(subset=['asin'], keep=False)]

,asin,title,description,price,books_rank,related_also_bought,related_bought_together,related_bought_after_viewing,category


In [ ]:
print_basic_data_stats(merged_meta_left)

Data shape:
(2372747, 9)

Missing values in mergeddata:
asin                                  0
title                            396700
description                     1251300
price                            692992
books_rank                       479812
related_also_bought             1103528
related_bought_together         1825314
related_bought_after_viewing    1127116
category                        1942372
dtype: int64

Percentage of missing values in metadata:
asin                             0.000000
title                           16.719018
description                     52.736343
price                           29.206317
books_rank                      20.221794
related_also_bought             46.508456
related_bought_together         76.928303
related_bought_after_viewing    47.502578
category                        81.861741
dtype: float64


In this code cell we merge the 2014 and 2018 metadata together, and we fill in the category and title values that are missing from the 2014 dataset, with the ones that are present in the 2018 dataset.

In [ ]:
# Deduplicate meta_data_raw_2018 based on 'asin', keeping the first occurrence
meta_data_raw_2018 = meta_data_raw_2018[['asin', 'title', 'category']].drop_duplicates(subset=['asin'], keep='first')
# 1. Merge with selected columns
merged_meta = pd.merge(book_meta_data, meta_data_raw_2018[['asin', 'title', 'category']],
                       on='asin', how='left', suffixes=('', '_2018'))

# 2. Fill missing values for specific columns
merged_meta['title'] = merged_meta['title'].fillna(merged_meta['title_2018'])
merged_meta['category'] = merged_meta['category'].fillna(merged_meta['category_2018'])

# 3. Drop temporary columns
merged_meta = merged_meta.drop(columns=['title_2018', 'category_2018'])

In [ ]:
print_basic_data_stats(merged_meta)

Data shape:
(2370585, 9)

Missing values in mergeddata:
asin                                  0
title                            396700
description                     1249227
price                            691175
books_rank                       479568
related_also_bought             1101588
related_bought_together         1823156
related_bought_after_viewing    1125260
category                         571138
dtype: int64

Percentage of missing values in metadata:
asin                             0.000000
title                           16.734266
description                     52.696993
price                           29.156305
books_rank                      20.229943
related_also_bought             46.469036
related_bought_together         76.907430
related_bought_after_viewing    47.467608
category                        24.092703
dtype: float64


In this code cell we repeat the merging process but instead, merge the 2014 and 2023 metadata together, and fill in the category and title values that are missing from the 2014 dataset, with the ones that are present in the 2023 dataset.

In [ ]:
# 1. Merge with selected columns
merged_meta = pd.merge(merged_meta, meta_data_raw_2023[['asin', 'title', 'category']],
                       on='asin', how='left', suffixes=('', '_2023'))

# 2. Fill missing values for specific columns
merged_meta['title'] = merged_meta['title'].fillna(merged_meta['title_2023'])
merged_meta['category'] = merged_meta['category'].fillna(merged_meta['category_2023'])

# 3. Drop temporary columns
merged_meta = merged_meta.drop(columns=['title_2023', 'category_2023'])

In [ ]:
print_basic_data_stats(merged_meta)

Data shape:
(2370585, 9)

Missing values in mergeddata:
asin                                  0
title                            382340
description                     1249227
price                            691175
books_rank                       479568
related_also_bought             1101588
related_bought_together         1823156
related_bought_after_viewing    1125260
category                         353832
dtype: int64

Percentage of missing values in metadata:
asin                             0.000000
title                           16.128508
description                     52.696993
price                           29.156305
books_rank                      20.229943
related_also_bought             46.469036
related_bought_together         76.907430
related_bought_after_viewing    47.467608
category                        14.925936
dtype: float64


In [ ]:
book_meta_data.shape

(2370585, 9)

# Merge the Book Review and Meta Datasets together

In [ ]:
merged_data = pd.merge(book_review_data, merged_meta, on='asin', how='inner')

In [ ]:
merged_data.shape

(8898041, 18)

In [ ]:
book_review_data.shape

(8898041, 10)

In [ ]:
print_basic_data_stats(merged_data)

Data shape:
(8898041, 18)

Missing values in mergeddata:
reviewerID                            0
asin                                  0
overall                               0
unixReviewTime                        0
user_id                               0
item_id                               0
total_user_reviews                    0
average_rating                        0
helpful_votes                         0
total_votes                           0
title                           1315467
description                     2018092
price                            139547
books_rank                      1506917
related_also_bought              858586
related_bought_together         4262693
related_bought_after_viewing     943757
category                         228991
dtype: int64

Percentage of missing values in metadata:
reviewerID                       0.000000
asin                             0.000000
overall                          0.000000
unixReviewTime                   0.000000

Now that we merged everything together we only have around 2.5% missing values!

In [ ]:
check_memory_usage(merged_data)

Total memory usage: 17002.52 MB


In [ ]:
merged_data.dtypes

,0
reviewerID,object
asin,object
overall,float64
unixReviewTime,int64
user_id,int64
item_id,int64
total_user_reviews,int64
average_rating,float64
helpful_votes,int64
total_votes,int64


# Optimize columns before saving

In [ ]:
def optimize_data_columns(df):
  # Optimize DataFrame columns
  df['reviewerID'] = df['reviewerID'].astype('category')
  df['asin'] = df['asin'].astype('category')
  df['user_id'] = pd.to_numeric(df['user_id'], downcast='integer')
  df['item_id'] = pd.to_numeric(df['item_id'], downcast='integer')
  df['helpful_votes'] = pd.to_numeric(df['helpful_votes'], downcast='integer')
  df['total_votes'] = pd.to_numeric(df['total_votes'], downcast='integer')
  df['title'] = df['title'].astype('category')
  df['category'] = df['category'].astype('category')
  df['total_user_reviews'] = pd.to_numeric(df['total_user_reviews'], downcast='integer')
  df['average_rating'] = pd.to_numeric(df['average_rating'], downcast='float')
  df['price'] = pd.to_numeric(df['price'], downcast='float')
  df['overall'] = pd.to_numeric(df['overall'], downcast='float')
  df['unixReviewTime'] = pd.to_numeric(df['unixReviewTime'], downcast='integer')
  df['books_rank'] = pd.to_numeric(df['books_rank'], downcast='integer')
  df['price'] = pd.to_numeric(df['price'], downcast='float')
  return df

In [ ]:
merged_data = optimize_data_columns(merged_data)

In [ ]:
check_memory_usage(merged_data)

Total memory usage: 14277.57 MB


In [ ]:
merged_data.dtypes

,0
reviewerID,category
asin,category
overall,float32
unixReviewTime,int32
user_id,int32
item_id,int32
total_user_reviews,int16
average_rating,float32
helpful_votes,int16
total_votes,int16


In [ ]:
dataset_all_features = merged_data.copy()

In [ ]:
dataset_all_features.to_csv('/content/drive/MyDrive/IR/dataset_all_features.csv.gz', compression='gzip', index=False)

In [ ]:
dataset_only_essential_features = merged_data[['user_id', 'item_id', 'overall', 'unixReviewTime', 'total_user_reviews', 'average_rating', 'title', 'category', 'price', 'books_rank']].copy()

In [ ]:
dataset_only_essential_features.head()

,user_id,item_id,overall,unixReviewTime,total_user_reviews,average_rating,title,category,price,books_rank
0,0,0,5.0,1355616000,8,4.375000,The Prophet,Literature & Fiction,3.99,587803
1,1,0,5.0,1071100800,556,4.201439,The Prophet,Literature & Fiction,3.99,587803
2,2,0,5.0,1390003200,6,4.333333,The Prophet,Literature & Fiction,3.99,587803
3,3,0,5.0,1317081600,9,4.000000,The Prophet,Literature & Fiction,3.99,587803
4,4,0,5.0,1033948800,76,3.750000,The Prophet,Literature & Fiction,3.99,587803


In [ ]:
print_value_counts(dataset_only_essential_features, 'category')

category
Literature & Fiction                              1996951
Literature &amp; Fiction                          1204330
Romance                                            466971
Children's Books                                   421015
Mystery, Thriller &amp; Suspense                   408818
Mystery, Thriller & Suspense                       368796
Christian Books &amp; Bibles                       302463
Christian Books & Bibles                           251663
Science Fiction &amp; Fantasy                      222012
History                                            216474
Teen & Young Adult                                 200732
Biographies & Memoirs                              172937
Teen &amp; Young Adult                             170951
Biographies &amp; Memoirs                          162663
Science Fiction & Fantasy                          154344
Self-Help                                          109774
Business & Money                                    95934
Polit

In [ ]:
dataset_only_essential_features.to_csv('/content/drive/MyDrive/IR/dataset_only_essential_features.csv.gz', compression='gzip', index=False)